## Retriever and Chain with Langchain

In [ ]:
## pdf reader
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('attention.pdf')
docs=loader.load()  
docs

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunk_documents=text_splitter.split_documents(docs)
chunk_documents

In [ ]:
## Vector embedding and storage
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

db=FAISS.from_documents(chunk_documents,OpenAIEmbeddings())
db

In [ ]:
query="Explain Transformers."
retrieved_result=db.similarity_search(query,k=2)
print(retrieved_result[0].page_content)

In [ ]:
## Design ChatPrompt Template
from langchain.prompts.chat import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.
<context>
{context}
</context>

Question : {input}

""")


In [ ]:
from langchain_community.llms import Ollama
## Ollama llama2
llm=Ollama(model="llama2")
llm

In [ ]:
## Chain
from langchain.chains.combine_documents.summarize import create_stuff_document_chain
doc_chain = create_stuff_document_chain(llm=llm, prompt=prompt)
doc_chain



In [ ]:
"""
Retrievers : A retriever is a component that fetches relevant documents from a knowledge base based on a user's query. 
It uses techniques like keyword matching or semantic search to find the most pertinent information.

"""
retriever=db.as_retriever()
retriever

In [ ]:
"""
Retrieval Chain: A Retrieval Chain is a chain that takes a user query, 
passes it to a retriever to fetch relevant documents, 
and then passes both the query and the retrieved documents to an LLM to generate an answer.
This is the core mechanism for Retrieval-Augmented Generation (RAG) in LangChain.

"""
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,doc_chain)


In [ ]:
response=retrieval_chain.invoke({"input":"Explain Transformers."})
response['answer']